In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/test/test/testX.pt
/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/train/train/trainY.pt
/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/train/train/trainX.pt


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy
import torchvision.models as m
from torchvision import datasets, transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import numpy as np
import pickle as pkl
from torchvision.utils import save_image
import cv2
from sklearn.metrics import mean_squared_error, r2_score
import pickle
import pandas as pd

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
torch.cuda.is_available()

True

In [5]:
trainx = torch.load('/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/train/train/trainX.pt')
trainy = torch.load('/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/train/train/trainY.pt')
testx = torch.load('/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/test/test/testX.pt')

num_train = trainx[0].shape[0]
num_test = testx[0].shape[0]

os.makedirs('./lazydata', exist_ok=True)

# Save train data
os.makedirs('./lazydata/train', exist_ok=True)
os.makedirs('./lazydata/train/X', exist_ok=True)
os.makedirs('./lazydata/train/Y', exist_ok=True)
for i in range(num_train):
    os.makedirs('./lazydata/train/X/{}'.format(i), exist_ok=True)
    # rgb
    os.makedirs('./lazydata/train/X/{}/rgb'.format(i), exist_ok=True)
    for j in range(3):
        save_image(trainx[0][i][j]/255, './lazydata/train/X/{}/rgb/{}.png'.format(i, j))
    # depth
    depth = trainx[1][i].numpy()
    np.save('./lazydata/train/X/{}/depth.npy'.format(i), depth)
    # field id
    pkl.dump(trainx[2][i], open('./lazydata/train/X/{}/field_id.pkl'.format(i), 'wb'))

    y = trainy[0][i].numpy()
    np.save('./lazydata/train/Y/{}.npy'.format(i), y)
print("Saved train data")

# Save test data
os.makedirs('./lazydata/test', exist_ok=True)
os.makedirs('./lazydata/test/X', exist_ok=True)
for i in range(num_test):
    os.makedirs('./lazydata/test/X/{}'.format(i), exist_ok=True)
    # rgb
    os.makedirs('./lazydata/test/X/{}/rgb'.format(i), exist_ok=True)
    for j in range(3):
        save_image(testx[0][i][j]/255, './lazydata/test/X/{}/rgb/{}.png'.format(i, j))
    # depth
    depth = testx[1][i].numpy()
    np.save('./lazydata/test/X/{}/depth.npy'.format(i), depth)
    # field id
    pkl.dump(testx[2][i], open('./lazydata/test/X/{}/field_id.pkl'.format(i), 'wb'))

print("Saved test data")

Saved train data
Saved test data


In [38]:
class lazyloaddata(Dataset):
    def __init__(self,path,train=True,transform=None):
        self.transform = transform
        path1 = path + ('train/' if train else 'test/')
        
        self.pathX = path1+'X/'
        self.pathY = path+'train/Y/'
        
        self.data = os.listdir(self.pathX)
        
    def __getitem__(self,idx):
        f = self.data[idx]
        img0 = cv2.imread(self.pathX+f+'/rgb/0.png')
        img1 = cv2.imread(self.pathX+f+'/rgb/1.png')
        img2 = cv2.imread(self.pathX+f+'/rgb/2.png')
        if self.transform is not None:
            '''
            img0 = cv2.normalize(img0,None,alpha=0,beta=1,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            img1 = cv2.normalize(img1,None,alpha=0,beta=1,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            img2 = cv2.normalize(img2,None,alpha=0,beta=1,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            '''
            
            img0 = self.transform(img0)
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        depth = np.load(self.pathX+f+'/depth.npy')
        
        fieldid = pkl.load(open(self.pathX+f+'/field_id.pkl','rb'))
        
        Y = np.load(self.pathY+f+'.npy')
        Y*=1000
        
        return (img0,img1,img2,depth,fieldid), Y
    
    def __len__(self):
        return len(self.data)

In [7]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

In [8]:
train_dataset = lazyloaddata('/kaggle/working/lazydata/', train = True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [9]:
mean = 0
count = 0
mean1 = 0
mean2 = 0
std = 0
std1 = 0
std2 = 0
for batch_id , ((img0,img1,img2,depth,fieldid), Y ) in enumerate(train_loader):
    mean+=img0.mean()
    mean1+=img1.mean()
    mean2+=img2.mean()
    std+=img0.std()
    std1+=img1.std()
    std2+=img2.std()
    count=count+1
print(mean/count)
print(mean1/count)
print(mean2/count)
print(std/count)
print(std1/count)
print(std2/count)

tensor(0.4159)
tensor(0.4860)
tensor(0.4806)
tensor(0.2044)
tensor(0.2327)
tensor(0.2379)


In [39]:
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(224),
    transforms.RandomInvert(),
    #transforms.RandomRotation((-90,90)),
    #transforms.RandomGrayscale(0.1),
    #transforms.GaussianBlur(kernel_size=7),
    #transforms.RandomRotation(30), #Just added
    transforms.ToTensor(),
    transforms.Normalize(mean = [0.4159, 0.4860, 0.4806], std = [0.2044, 0.2327, 0.2379])
])

In [40]:
train_dataset = lazyloaddata('/kaggle/working/lazydata/',transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [42]:
res18 = m.resnet50()
'''
resnet50(weights=None)
resnet50()
resnet50(pretrained=False)  # deprecated
resnet50(False)  # deprecated
'''


res18.train()
res18.float()
res18.conv1 = nn.Conv2d(12,64,kernel_size=7,stride=2,padding=3,bias=False)
res18.fc = nn.Linear(in_features=2048, out_features=12) 

weight = res18.conv1.weight.clone()

'''
From the website:
https://stackoverflow.com/questions/62629114/how-to-modify-resnet-50-with-4-channels-as-input-using-pre-trained-weights-in-py
'''
with torch.no_grad():
    res18.conv1.weight[:,:12] = weight
    res18.conv1.weight[:, 3] = res18.conv1.weight[:, 0]





In [43]:
model = res18

In [44]:
optimizer = torch.optim.Adam(res18.parameters())
#optimizer = torch.optim.Adam(res18.parameters(), lr = 0.00001)

In [45]:
def train(epoch, model, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # send to device
        data = torch.cat((data[0],data[1],data[2],data[3]),1)
        data, target = data.to(device), target.to(device)
        model = model.to(device)
        output = model(data)
        criterion = nn.MSELoss()
        #loss = criterion(output,target)
        loss = criterion(output.float(),target.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            
            
'''
def test(model, permute_pixels=None, permutation_order=None):
    model.eval()
    test_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        RGBs=torch.stack((data[0][:,:,:,0],data[0][:,:,:,1],data[0][:,:,:,2],
                          data[1][:,:,:,0],data[1][:,:,:,1],data[1][:,:,:,2],
                          data[2][:,:,:,0],data[2][:,:,:,1],data[2][:,:,:,2]),1)
        data=torch.cat((RGBs,data[3]), 1)
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        criterion = nn.MSELoss()
        loss = criterion(output.float(),target.float())
        test_loss += loss# sum up batch loss
        #print(output.shape)
        #print(output)
        #pred = output.data.max(1)[1]  # get the index of the max log-probability 
        #print(pred.shape)
        #print(pred)
        #print(target.shape)
        #print(target)
        #correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        print(output)
        print(target)
        correct += np.sqrt(mean_squared_error(output.data, target.data))
    correct /= len(train_loader.dataset)
    accuracy = correct
    print(accuracy)
    return accuracy
'''            


'\ndef test(model, permute_pixels=None, permutation_order=None):\n    model.eval()\n    test_loss = 0\n    correct = 0\n    for batch_idx, (data, target) in enumerate(train_loader):\n        RGBs=torch.stack((data[0][:,:,:,0],data[0][:,:,:,1],data[0][:,:,:,2],\n                          data[1][:,:,:,0],data[1][:,:,:,1],data[1][:,:,:,2],\n                          data[2][:,:,:,0],data[2][:,:,:,1],data[2][:,:,:,2]),1)\n        data=torch.cat((RGBs,data[3]), 1)\n        data, target = data.to(device), target.to(device)\n        optimizer.zero_grad()\n        output = model(data)\n        criterion = nn.MSELoss()\n        loss = criterion(output.float(),target.float())\n        test_loss += loss# sum up batch loss\n        #print(output.shape)\n        #print(output)\n        #pred = output.data.max(1)[1]  # get the index of the max log-probability \n        #print(pred.shape)\n        #print(pred)\n        #print(target.shape)\n        #print(target)\n        #correct += pred.eq(target.

In [49]:
model = model.to(device)
num_epochs = 80
#45
for i in range(1,num_epochs+1):
    train(i, model, optimizer)

Train Epoch: 1 [0/3396 (0%)]	Loss: 8.466755
Train Epoch: 1 [640/3396 (19%)]	Loss: 6.351728
Train Epoch: 1 [1280/3396 (37%)]	Loss: 2.809416
Train Epoch: 1 [1920/3396 (56%)]	Loss: 3.993989
Train Epoch: 1 [2560/3396 (74%)]	Loss: 3.975586
Train Epoch: 1 [3200/3396 (93%)]	Loss: 5.377496
Train Epoch: 2 [0/3396 (0%)]	Loss: 5.407528
Train Epoch: 2 [640/3396 (19%)]	Loss: 4.277002
Train Epoch: 2 [1280/3396 (37%)]	Loss: 3.180958
Train Epoch: 2 [1920/3396 (56%)]	Loss: 4.359470
Train Epoch: 2 [2560/3396 (74%)]	Loss: 5.279737
Train Epoch: 2 [3200/3396 (93%)]	Loss: 3.524143
Train Epoch: 3 [0/3396 (0%)]	Loss: 5.514608
Train Epoch: 3 [640/3396 (19%)]	Loss: 4.366031
Train Epoch: 3 [1280/3396 (37%)]	Loss: 7.926670
Train Epoch: 3 [1920/3396 (56%)]	Loss: 5.597160
Train Epoch: 3 [2560/3396 (74%)]	Loss: 5.236538
Train Epoch: 3 [3200/3396 (93%)]	Loss: 4.426623
Train Epoch: 4 [0/3396 (0%)]	Loss: 6.666349
Train Epoch: 4 [640/3396 (19%)]	Loss: 8.127910
Train Epoch: 4 [1280/3396 (37%)]	Loss: 5.446822
Train Epoch:

In [16]:
'''
num_epochs = 1
#45
for i in range(1,num_epochs+1):
    train(i, model, optimizer)
'''

Train Epoch: 1 [0/3396 (0%)]	Loss: 4284.554199
Train Epoch: 1 [640/3396 (19%)]	Loss: 1563.265991
Train Epoch: 1 [1280/3396 (37%)]	Loss: 798.788940
Train Epoch: 1 [1920/3396 (56%)]	Loss: 572.848267
Train Epoch: 1 [2560/3396 (74%)]	Loss: 561.103760
Train Epoch: 1 [3200/3396 (93%)]	Loss: 503.927612


In [50]:
optimizer = torch.optim.Adam(res18.parameters(), lr = 0.00001)
model = model.to(device)
num_epochs = 10
for i in range(1,num_epochs+1):
    train(i, model, optimizer)

Train Epoch: 1 [0/3396 (0%)]	Loss: 4.280384
Train Epoch: 1 [640/3396 (19%)]	Loss: 4.098396
Train Epoch: 1 [1280/3396 (37%)]	Loss: 4.072707
Train Epoch: 1 [1920/3396 (56%)]	Loss: 2.591498
Train Epoch: 1 [2560/3396 (74%)]	Loss: 3.656499
Train Epoch: 1 [3200/3396 (93%)]	Loss: 3.536957
Train Epoch: 2 [0/3396 (0%)]	Loss: 4.079210
Train Epoch: 2 [640/3396 (19%)]	Loss: 2.027227
Train Epoch: 2 [1280/3396 (37%)]	Loss: 4.497044
Train Epoch: 2 [1920/3396 (56%)]	Loss: 3.530198
Train Epoch: 2 [2560/3396 (74%)]	Loss: 3.315868
Train Epoch: 2 [3200/3396 (93%)]	Loss: 3.634104
Train Epoch: 3 [0/3396 (0%)]	Loss: 2.828182
Train Epoch: 3 [640/3396 (19%)]	Loss: 2.889345
Train Epoch: 3 [1280/3396 (37%)]	Loss: 2.045414
Train Epoch: 3 [1920/3396 (56%)]	Loss: 1.971209
Train Epoch: 3 [2560/3396 (74%)]	Loss: 3.040334
Train Epoch: 3 [3200/3396 (93%)]	Loss: 4.448372
Train Epoch: 4 [0/3396 (0%)]	Loss: 2.568496
Train Epoch: 4 [640/3396 (19%)]	Loss: 2.256904
Train Epoch: 4 [1280/3396 (37%)]	Loss: 4.576478
Train Epoch:

In [56]:
optimizer = torch.optim.Adam(res18.parameters(), lr = 0.000001)
model = model.to(device)
num_epochs = 5
for i in range(1,num_epochs+1):
    train(i, model, optimizer)

Train Epoch: 1 [0/3396 (0%)]	Loss: 1.256971
Train Epoch: 1 [640/3396 (19%)]	Loss: 1.881082
Train Epoch: 1 [1280/3396 (37%)]	Loss: 1.502058
Train Epoch: 1 [1920/3396 (56%)]	Loss: 2.608703
Train Epoch: 1 [2560/3396 (74%)]	Loss: 2.696876
Train Epoch: 1 [3200/3396 (93%)]	Loss: 3.214560
Train Epoch: 2 [0/3396 (0%)]	Loss: 1.894058
Train Epoch: 2 [640/3396 (19%)]	Loss: 1.759058
Train Epoch: 2 [1280/3396 (37%)]	Loss: 3.499740
Train Epoch: 2 [1920/3396 (56%)]	Loss: 4.704468
Train Epoch: 2 [2560/3396 (74%)]	Loss: 4.144655
Train Epoch: 2 [3200/3396 (93%)]	Loss: 1.493994
Train Epoch: 3 [0/3396 (0%)]	Loss: 1.532415
Train Epoch: 3 [640/3396 (19%)]	Loss: 1.980862
Train Epoch: 3 [1280/3396 (37%)]	Loss: 2.995982
Train Epoch: 3 [1920/3396 (56%)]	Loss: 1.746257
Train Epoch: 3 [2560/3396 (74%)]	Loss: 3.740735
Train Epoch: 3 [3200/3396 (93%)]	Loss: 1.580100
Train Epoch: 4 [0/3396 (0%)]	Loss: 4.808509
Train Epoch: 4 [640/3396 (19%)]	Loss: 1.566144
Train Epoch: 4 [1280/3396 (37%)]	Loss: 2.611864
Train Epoch:

In [37]:
'''
optimizer = torch.optim.Adam(res18.parameters(), lr = 0.0000001)
model = model.to(device)
num_epochs = 5
for i in range(1,num_epochs+1):
    train(i, model, optimizer)
'''

'\noptimizer = torch.optim.Adam(res18.parameters(), lr = 0.0000001)\nmodel = model.to(device)\nnum_epochs = 5\nfor i in range(1,num_epochs+1):\n    train(i, model, optimizer)\n'

In [57]:
test_data = torch.load("/kaggle/input/csci-ua-473-intro-to-machine-learning-fall-2022/test/test/testX.pt")

In [58]:

'''
file_ids = test_data[-1]
depths = test_data[1]
rgbs = test_data[0]
'''


#test_dataset = lazyloaddata("/kaggle/working/lazydata/",train = False, transform = transform)
#test_dataset = lazyloaddata("/kaggle/working/lazydata/",train = False,transform = transform)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=True)

file_ids = test_data[-1]
depths = test_data[1]
rgbs = test_data[0]
test_img_data = torch.cat((rgbs[:,0],rgbs[:,1],rgbs[:,2],depths),dim=1)
split_test = torch.split(test_img_data,12,dim=0)
preds = []

model = model.to(device)
'''
for batch_idx, (data, target) in enumerate(test_loader):
        # send to device
        data = torch.cat((data[0],data[1],data[2],data[3]),1)
        data = data.to(device)
        output = model(data)/1000
        preds.append(output.cpu().detach().numpy())
'''
for data in split_test:
    output = model(data.to(device))/1000
    preds.append(output.cpu().detach().numpy())






In [59]:
outfile = 'submission.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
# preds = []

# test_data = torch.load('./test/test/testX.pt')
# file_ids = test_data[-1]
# rgb_data = test_data[0]
# model.eval()

# for i, data in enumerate(rgb_data):   
#     # Please remember to modify this loop, input and output based on your model/architecture
#     output = model(data[:1, :, :, :].to('cuda'))
#     preds.append(output[0].cpu().detach().numpy())

df = pd.concat([pd.DataFrame(file_ids), pd.DataFrame.from_records(np.concatenate(preds))], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Written to csv file submission.csv
